In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import os
from glob import glob

np.random.seed(5)

In [ ]:
img_dir1 = './data_origin/train'
categories1 = os.listdir(img_dir1)
nb_classes1 = len(categories1)

img_dir2 = './data_origin/test'
categories2 = os.listdir(img_dir2)
nb_classes2 = len(categories2)

print(categories1,'\n')
print(categories2)

In [ ]:
train_dir = os.path.join('./data_origin/train')
test_dir = os.path.join('./data_origin/test')

#train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.1)
train_datagen = ImageDataGenerator(
      rescale=1./255,      # 영상의 전체 픽셀을 255로 나눈다 - 영상이 0~1사이의 범위를 가지게 된다(normalization)
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(train_dir,
                                                   target_size=(720,576),
                                                   batch_size=100,
                                                   class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(test_dir,
                                                 target_size=(720,576),
                                                 batch_size=10,
                                                 class_mode='categorical')


In [ ]:
for data_batch, labels_batch in train_generator:
    print('배치 데이터 크기:', data_batch.shape)
    print('배치 레이블 크기:', labels_batch.shape)
    break

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3),
                activation='relu',
                input_shape=(720,576,3)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(8, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['acc'])

In [ ]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

In [ ]:
hist = model.fit_generator(train_generator,
                           steps_per_epoch=2000,
                           epochs=10,
                           validation_data=test_generator,
                           validation_steps=800)

In [ ]:
model.save_weights('first_try.h5')

In [ ]:
scores = model.evaluate_generator(test_generator, steps=10)
print(scores)

In [ ]:
out = model.predict_generator(test_generator, steps=10)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_generator.class_indices)
print(out)